In [1]:
import PyPDF2
import nltk
import pandas as pd
from nltk import word_tokenize

In [2]:
fed_20210924 = PyPDF2.PdfReader('Fed/20210924.pdf')
fed_20211109 = PyPDF2.PdfReader('Fed/20211109.pdf')
fed_20211129 = PyPDF2.PdfReader('Fed/20211129.pdf')
fed_20220321 = PyPDF2.PdfReader('Fed/20220321.pdf')
fed_20220524 = PyPDF2.PdfReader('Fed/20220524.pdf')
fed_20220617 = PyPDF2.PdfReader('Fed/20220617.pdf')
fed_20220826 = PyPDF2.PdfReader('Fed/20220826.pdf')
fed_20220928 = PyPDF2.PdfReader('Fed/20220928.pdf')
fed_20221130 = PyPDF2.PdfReader('Fed/20221130.pdf')
fed_20230110 = PyPDF2.PdfReader('Fed/20230110.pdf')

In [3]:

# Load sentiment words from files
def load_sentiment_words(file):
    with open(file, 'r') as f:
        words = [line.strip() for line in f.readlines()]
    return words

positive_words = load_sentiment_words('Sentiment/ML_positive_unigram.txt')
negative_words = load_sentiment_words('Sentiment/ML_negative_unigram.txt')

# Calculate sentiment score using custom sentiment word lists
def custom_sentiment_score(text):
    tokenized_text = word_tokenize(text)
    positive_count = sum([1 for word in tokenized_text if word.lower() in positive_words])
    negative_count = sum([1 for word in tokenized_text if word.lower() in negative_words])
    return positive_count - negative_count

# Read PDF files and calculate sentiment scores
announcement_files = {
    '20210924': 'Fed/20210924.pdf',
    '20211109': 'Fed/20211109.pdf',
    '20211129': 'Fed/20211129.pdf',
    '20220321': 'Fed/20220321.pdf',
    '20220524': 'Fed/20220524.pdf',
    '20220617': 'Fed/20220617.pdf',
    '20220826': 'Fed/20220826.pdf',
    '20220928': 'Fed/20220928.pdf',
    '20221130': 'Fed/20221130.pdf',
    '20230110': 'Fed/20230110.pdf',
}

sentiment_scores = {}

for date, file in announcement_files.items():
    with open(file, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        sentiment_scores[date] = custom_sentiment_score(text)

print(sentiment_scores)

{'20210924': 8, '20211109': 26, '20211129': 21, '20220321': 2, '20220524': 28, '20220617': 9, '20220826': 18, '20220928': 12, '20221130': 28, '20230110': 1}


In [4]:
# Convert sentiment_scores dictionary to DataFrame
sentiment_score_df = pd.DataFrame.from_dict(sentiment_scores, orient='index', columns=['SentimentScore'])

# Reset the index and rename the index column to 'Date'
sentiment_score_df = sentiment_score_df.reset_index().rename(columns={'index': 'Date'})

# Convert the 'Date' column to datetime
sentiment_score_df['Date'] = pd.to_datetime(sentiment_score_df['Date'])

# Print the resulting DataFrame
print(sentiment_score_df)


        Date  SentimentScore
0 2021-09-24               8
1 2021-11-09              26
2 2021-11-29              21
3 2022-03-21               2
4 2022-05-24              28
5 2022-06-17               9
6 2022-08-26              18
7 2022-09-28              12
8 2022-11-30              28
9 2023-01-10               1


In [5]:
sentiment_score_df.to_csv('analysis csv file/sentiment_score_df.csv', index=False)